In [1]:
import pandas as pd
import numpy as np
import math
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices, dmatrix, demo_data
from sklearn.preprocessing import MinMaxScaler

In [2]:
df= pd.read_csv("dataset.csv", encoding='cp949', index_col="Unnamed: 0")
df

,CPU1,CPU2,CPU3,GPU,크기,해상도,밀도,RAM,외장메모리,내장메모리,...,지문인식,S펜,빅스비,삼성Dex,삼성페이,주사율,폴더블,출시년도,소비자물가,출고가
Galaxy S,1.00,0.00,0.0,200,4.0,384000,235.1,0.5,1,8,...,0,0,0,0,0,30,0,2010,1101188,949300
Galaxy U,1.00,0.00,0.0,200,4.0,384000,235.1,0.5,1,8,...,0,0,0,0,0,30,0,2010,1042840,899000
Galaxy K,1.00,0.00,0.0,200,4.0,384000,235.1,0.5,1,8,...,0,0,0,0,0,30,0,2010,1042840,899000
Galaxy Note,1.50,0.00,0.0,266,5.3,1024000,285.5,1.0,1,16,...,0,1,0,0,0,60,0,2011,1067055,957000
Galaxy S Hoppin,1.00,0.00,0.0,200,4.0,384000,235.1,0.5,1,16,...,0,0,0,0,0,30,0,2011,1115000,1000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Galaxy S20 FE,2.84,2.42,1.8,587,6.5,2592000,407.0,6.0,1,128,...,1,0,1,1,1,120,0,2020,899800,899800
Galaxy Note 20,3.09,2.42,1.8,670,6.7,2592000,393.0,8.0,0,256,...,1,1,1,1,1,60,0,2020,1199000,1199000
Galaxy Note 20 Ultra,3.09,2.42,1.8,670,6.9,4446720,496.0,12.0,1,256,...,1,1,1,1,1,120,0,2020,1452000,1452000
Galaxy Z Flip,2.96,1.78,0.0,675,6.7,2846880,425.0,8.0,0,256,...,1,0,1,0,1,60,1,2020,1650000,1650000


In [3]:
#스피어만 상관계수: 정규성 깨졌을 경우, 순서형 자료, 자료에 이상치 있거나 표본 크기 작을 때
spearman = df.corr(method='spearman')
spearman

,CPU1,CPU2,CPU3,GPU,크기,해상도,밀도,RAM,외장메모리,내장메모리,...,지문인식,S펜,빅스비,삼성Dex,삼성페이,주사율,폴더블,출시년도,소비자물가,출고가
CPU1,1.000000,0.929527,0.727540,0.565246,0.891140,0.733826,0.362739,0.936292,-0.332606,0.871992,...,0.768276,0.008708,0.827815,0.782938,0.770157,0.604066,0.336214,0.973074,0.520614,0.661648
CPU2,0.929527,1.000000,0.759965,0.619670,0.909628,0.797394,0.413105,0.963185,-0.302682,0.906895,...,0.734657,0.055976,0.871824,0.860895,0.834286,0.574761,0.245070,0.972200,0.570005,0.704997
CPU3,0.727540,0.759965,1.000000,0.526849,0.672303,0.542708,0.046927,0.749381,-0.267613,0.702642,...,0.383768,-0.050022,0.686505,0.707872,0.458335,0.385724,0.112763,0.732601,0.536698,0.589370
GPU,0.565246,0.619670,0.526849,1.000000,0.536772,0.609680,0.582500,0.645013,-0.371556,0.642869,...,0.748722,0.120640,0.338043,0.320660,0.697298,0.277078,0.116807,0.615560,0.377241,0.496315
크기,0.891140,0.909628,0.672303,0.536772,1.000000,0.773399,0.265486,0.934583,-0.382740,0.885189,...,0.688826,0.268228,0.865270,0.828702,0.710803,0.560493,0.338986,0.912747,0.650097,0.776010
해상도,0.733826,0.797394,0.542708,0.609680,0.773399,1.000000,0.699909,0.819119,-0.047053,0.785755,...,0.770249,0.091775,0.714746,0.730832,0.757864,0.500647,-0.028926,0.790529,0.525177,0.657542
밀도,0.362739,0.413105,0.046927,0.582500,0.265486,0.699909,1.000000,0.400006,0.159875,0.407382,...,0.716851,-0.088352,0.239839,0.260672,0.662501,0.241720,-0.193020,0.419229,0.063676,0.207208
RAM,0.936292,0.963185,0.749381,0.645013,0.934583,0.819119,0.400006,1.000000,-0.369472,0.916789,...,0.749943,0.143256,0.841687,0.812883,0.816371,0.573452,0.294864,0.963863,0.606431,0.738999
외장메모리,-0.332606,-0.302682,-0.267613,-0.371556,-0.382740,-0.047053,0.159875,-0.369472,1.000000,-0.332735,...,-0.258775,-0.089165,-0.208310,-0.136745,-0.309055,-0.074758,-0.516398,-0.323732,-0.295046,-0.317353
내장메모리,0.871992,0.906895,0.702642,0.642869,0.885189,0.785755,0.407382,0.916789,-0.332735,1.000000,...,0.736662,0.096472,0.845873,0.814521,0.821338,0.436577,0.274763,0.908330,0.680698,0.798956


In [4]:
spearman = spearman [abs(spearman) >= 0.01].stack().reset_index()
spearman  = spearman [spearman ['level_0'].astype(str)!=spearman['level_1'].astype(str)]

spearman ['ordered-cols'] = spearman.apply(lambda x: '-'.join(sorted([x['level_0'],x['level_1']])),axis=1)
spearman  = spearman.drop_duplicates(['ordered-cols'])
spearman.drop(['ordered-cols'], axis=1, inplace=True)
 
spearman.sort_values(by=[0], ascending=False).head(50)

,level_0,level_1,0
27,CPU1,출시년도,0.973074
58,CPU2,출시년도,0.972200
75,CPU3,후면광각,0.971302
488,전면카메라화소,출시년도,0.970056
726,빅스비,삼성Dex,0.969815
45,CPU2,전면카메라화소,0.966695
917,소비자물가,출고가,0.966258
244,RAM,출시년도,0.963863
37,CPU2,RAM,0.963185
231,RAM,전면카메라화소,0.952055


In [5]:
minmax = MinMaxScaler()
fitted =  minmax.fit(df)
output = minmax.transform(df)
output = pd.DataFrame(output, columns=df.columns, index=list(df.index.values))
output

,CPU1,CPU2,CPU3,GPU,크기,해상도,밀도,RAM,외장메모리,내장메모리,...,지문인식,S펜,빅스비,삼성Dex,삼성페이,주사율,폴더블,출시년도,소비자물가,출고가
Galaxy S,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046518,0.000000,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.279208,0.216071
Galaxy U,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046518,0.000000,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.247191,0.188853
Galaxy K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046518,0.000000,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.247191,0.188853
Galaxy Note,0.239234,0.000000,0.000000,0.115385,0.361111,0.151515,0.186908,0.043478,1.0,0.007874,...,0.0,1.0,0.0,0.0,0.0,0.333333,0.0,0.1,0.260478,0.220238
Galaxy S Hoppin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046518,0.000000,1.0,0.007874,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.286786,0.243506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Galaxy S20 FE,0.880383,1.000000,0.923077,0.676573,0.694444,0.522727,0.525348,0.478261,1.0,0.118110,...,1.0,0.0,1.0,1.0,1.0,1.000000,0.0,1.0,0.168703,0.189286
Galaxy Note 20,1.000000,1.000000,0.923077,0.821678,0.750000,0.522727,0.486351,0.652174,0.0,0.244094,...,1.0,1.0,1.0,1.0,1.0,0.333333,0.0,1.0,0.332879,0.351190
Galaxy Note 20 Ultra,1.000000,1.000000,0.923077,0.821678,0.805556,0.961818,0.773259,1.000000,1.0,0.244094,...,1.0,1.0,1.0,1.0,1.0,1.000000,0.0,1.0,0.471704,0.488095
Galaxy Z Flip,0.937799,0.735537,0.000000,0.830420,0.750000,0.583068,0.575487,0.652174,0.0,0.244094,...,1.0,0.0,1.0,0.0,1.0,0.333333,1.0,1.0,0.580350,0.595238


In [6]:
df = output
df

,CPU1,CPU2,CPU3,GPU,크기,해상도,밀도,RAM,외장메모리,내장메모리,...,지문인식,S펜,빅스비,삼성Dex,삼성페이,주사율,폴더블,출시년도,소비자물가,출고가
Galaxy S,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046518,0.000000,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.279208,0.216071
Galaxy U,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046518,0.000000,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.247191,0.188853
Galaxy K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046518,0.000000,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.247191,0.188853
Galaxy Note,0.239234,0.000000,0.000000,0.115385,0.361111,0.151515,0.186908,0.043478,1.0,0.007874,...,0.0,1.0,0.0,0.0,0.0,0.333333,0.0,0.1,0.260478,0.220238
Galaxy S Hoppin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046518,0.000000,1.0,0.007874,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.286786,0.243506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Galaxy S20 FE,0.880383,1.000000,0.923077,0.676573,0.694444,0.522727,0.525348,0.478261,1.0,0.118110,...,1.0,0.0,1.0,1.0,1.0,1.000000,0.0,1.0,0.168703,0.189286
Galaxy Note 20,1.000000,1.000000,0.923077,0.821678,0.750000,0.522727,0.486351,0.652174,0.0,0.244094,...,1.0,1.0,1.0,1.0,1.0,0.333333,0.0,1.0,0.332879,0.351190
Galaxy Note 20 Ultra,1.000000,1.000000,0.923077,0.821678,0.805556,0.961818,0.773259,1.000000,1.0,0.244094,...,1.0,1.0,1.0,1.0,1.0,1.000000,0.0,1.0,0.471704,0.488095
Galaxy Z Flip,0.937799,0.735537,0.000000,0.830420,0.750000,0.583068,0.575487,0.652174,0.0,0.244094,...,1.0,0.0,1.0,0.0,1.0,0.333333,1.0,1.0,0.580350,0.595238


In [8]:
model2 = smf.ols(formula = '출고가~크기+RAM+후면카메라+두께+폴더블+출시년도', data = df)
result2 = model2.fit() 
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    출고가   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.822
Method:                 Least Squares   F-statistic:                     51.13
Date:                Sat, 07 Aug 2021   Prob (F-statistic):           1.44e-21
Time:                        23:51:58   Log-Likelihood:                 80.700
No. Observations:                  66   AIC:                            -147.4
Df Residuals:                      59   BIC:                            -132.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2020      0.045      4.503      0.000       0.112       0.292
크기             0.3339      0.102      3.261      0.002       0.129       0.539
RAM            0.1915      0.106      1.802      0.077      -0.021       0.404
후면카메라          0.1900      0.078      2.442      0.018       0.034       0.346
두께            -0.0763      0.081     -0.942      0.350      -0.238       0.086
폴더블            0.3834      0.051      7.522      0.000       0.281       0.485
출시년도          -0.2970      0.082     -3.608      0.001      -0.462      -0.132
==============================================================================
Omnibus:                        7.629   Durbin-Watson:                   1.830
Prob(Omnibus):                  0.022   Jarque-Bera (JB):                7.844
Skew:                           0.558   Prob(JB):                       0.0198
Kurtosis:                       4.268   Cond. No.                         19.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""